## Imports & Document Collection
This cell imports necessary libraries and defines the document collection for information retrieval experiments.


In [6]:
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from scipy.stats import entropy

# Document collection for IR
documents = [
    "Cats are small domesticated carnivorous mammals",
    "Artificial intelligence helps computers learn from data",
    "Natural language processing enables machines to understand text",
    "Search engines retrieve relevant web pages for queries",
    "Text analysis extracts insights from written content",
    "Neural networks are the backbone of deep learning models",
    "Data analytics combines math, programming, and domain knowledge"
]


## Boolean Retrieval
This function retrieves documents where **all query terms are present**.


In [7]:
def boolean_search(query, docs):
    query_terms = set(query.lower().split())
    results = []
    for idx, doc in enumerate(docs):
        doc_terms = set(re.findall(r"\w+", doc.lower()))
        if query_terms.issubset(doc_terms):
            results.append((idx, doc))
    return results


## TF-IDF Cosine Similarity
Compute similarity between query and documents using TF-IDF vectors and cosine similarity.


In [8]:
def tfidf_cosine_similarity(query, docs):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(docs + [query])
    query_vec = X[-1]
    doc_vecs = X[:-1]
    scores = [(i, 1 - cosine(query_vec.toarray().ravel(), doc_vecs[i].toarray().ravel()))
              for i in range(len(docs))]
    return sorted(scores, key=lambda x: -x[1])


## KL Divergence Similarity
Compute similarity based on KL divergence between the query and each document.


In [9]:
def kl_divergence_similarity(query, docs):
    vocab = list(set(" ".join(docs + [query]).lower().split()))
    
    def text_distribution(text):
        words = text.lower().split()
        counts = Counter(words)
        dist = np.array([counts[w] for w in vocab], dtype=float)
        dist = (dist + 1) / (dist.sum() + len(vocab))  # Laplace smoothing
        return dist

    q_dist = text_distribution(query)
    results = []
    for idx, doc in enumerate(docs):
        d_dist = text_distribution(doc)
        div = entropy(q_dist, d_dist)
        sim = 1 / (1 + div)
        results.append((idx, sim))
    return sorted(results, key=lambda x: -x[1])


## Run Models
Try a query and see results from all three models.


In [10]:
query = "artificial intelligence data"
print("Query:", query)

print("\nBoolean Retrieval:")
for idx, doc in boolean_search(query, documents):
    print(f"Doc {idx}: {doc}")

print("\nTF-IDF Cosine Similarity:")
for idx, score in tfidf_cosine_similarity(query, documents):
    print(f"Doc {idx}: {documents[idx]} | Score = {score:.4f}")

print("\nKL Divergence Similarity:")
for idx, score in kl_divergence_similarity(query, documents):
    print(f"Doc {idx}: {documents[idx]} | Score = {score:.4f}")


Query: artificial intelligence data

Boolean Retrieval:
Doc 1: Artificial intelligence helps computers learn from data

TF-IDF Cosine Similarity:
Doc 1: Artificial intelligence helps computers learn from data | Score = 0.5851
Doc 6: Data analytics combines math, programming, and domain knowledge | Score = 0.1373
Doc 0: Cats are small domesticated carnivorous mammals | Score = 0.0000
Doc 2: Natural language processing enables machines to understand text | Score = 0.0000
Doc 3: Search engines retrieve relevant web pages for queries | Score = 0.0000
Doc 4: Text analysis extracts insights from written content | Score = 0.0000
Doc 5: Neural networks are the backbone of deep learning models | Score = 0.0000

KL Divergence Similarity:
Doc 1: Artificial intelligence helps computers learn from data | Score = 0.9796
Doc 6: Data analytics combines math, programming, and domain knowledge | Score = 0.9507
Doc 0: Cats are small domesticated carnivorous mammals | Score = 0.9469
Doc 4: Text analysis e